# <font color='navy'> <b> Vytvorenie simulačného modelu pomocou knižnice gmsh-api </b></font>

Pre parametrické generovanie popisu problému v Pythone je možné použiť

* **gmsh-api** knižnica s priamym mapovaním funkcií *gmsh* do Pythonu
* **pygmsh** knižnica určené pre tvorbu 3D modelov využívajúca pokročilé konštrukcie z *OpenCASCADE* 

Pre 2D problémy, vyžadujúce definovanie orientovaných povrchov ako napr. elektród sa knižnica *pygmsh* ukázala ako nevhodná, je primárne orientovaná na popis 3D problémov a získať obrys 2D objektu je problematické. 

### <font color='teal'> <b> Použitie gmsh-api </b></font>

Knižnica obsahuje inicializáciu prostredia a pomocné funkcie pre zjednodušenie generovania základných konštrukcií. Globálny parameter **lc** určuje hustotou vygenerovanej mriežky. Princíp knižnice je zrejmý z nasledujúceho fragmentu programu.  

```Python
import gmsh
import numpy
import warnings; warnings.filterwarnings("ignore")

gmsh.initialize()
#gmsh.option.setNumber("General.Terminal",0)     # potlacenie vypisu a chyb
#gmsh.option.setNumber("General.Terminal", 1)

def Rectangle(x,y, w, h, lc=0.1):
    '''
    Funkcia pre vytorenie pravouholnika
    x,y - stred
    w,h - sirka, vyska
    
    p0---------l0------>--p1 
    ^                     |
    |                     |
    l3        (x,y)       l1
    |                     |
    |                     v
    p3--<------l2---------p2
    
    return [p0, p1, p2, p3], [l0, l1, l2, l3], c
    '''
    p0 = gmsh.model.geo.addPoint(x-w/2, y+h/2, 0, lc)
    p1 = gmsh.model.geo.addPoint(x+w/2, y+h/2, 0, lc)
    p2 = gmsh.model.geo.addPoint(x+w/2, y-h/2, 0, lc)
    p3 = gmsh.model.geo.addPoint(x-w/2, y-h/2, 0, lc)
    
    l0 = gmsh.model.geo.addLine(p0,p1)
    l1 = gmsh.model.geo.addLine(p1,p2)
    l2 = gmsh.model.geo.addLine(p2,p3)
    l3 = gmsh.model.geo.addLine(p3,p0)
    
    c = gmsh.model.geo.addCurveLoop([p0, p1, p2, p3])
    
    return ([p0,p1,p2,p3], [l0, l1,l2, l3], c)

def Point(x,y, lc=0.1):
    '''
    Vytvorenie bodu
    '''
    return gmsh.model.geo.addPoint(x, y, 0, lc)
    
def Line(pa, pb):
    '''
    Vytvorenie usecky
    '''
    return gmsh.model.geo.addLine(pa, pb)

```

### <font color='purple'> <b>  Príklad 1. Vytvorenie prázdneho objektu  </b></font>

Ak pri vytvarani povrchu zadáme v poradí vonkajší objekt a potom vnútorný, vnútorný bude pokladaný za vnútornú hranicu.  

In [1]:
%reset -sf 
from utils.utils import *
gmsh.option.setNumber("General.Terminal",0)

gmsh.model.add("Deravy model")
lc=0.1                                  # hustota generovanej mriezky
P0, L0, C0 = Rectangle(0,0, 10, 5)      # vonkajsi priestor
P1, L1, C1 = Rectangle(0,0, 3, 1)       # vnutorny priestor

gmsh.model.geo.addPlaneSurface([C0, C1])

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)
      
gmsh.write("./data/demo_api_01.vtk")
gmsh.write("./data/demo_api_01.msh") 
gmsh.write("./data/demo_api_01.geo_unrolled")   # zapis geo - suboru

#import os
#_ = os.system('gmsh ./data/demo_api_01.msh &')

Vygenerovana mriezka modelu

<img src="./img/obr_gmsh_01.png" width="600">

### <font color='purple'> <b>  Príklad 2. Vytvorenie vyplneného objektu  </b></font>

Ak **pred** vytváraním celkovej plochy vygenerujeme samostatne plochu z vnútorného objektu, tento bude renderovaný ako samostatný objekt.

In [2]:
%reset -sf 
from utils.utils import *
gmsh.option.setNumber("General.Terminal",0)

gmsh.model.add("Vyplnený model")            
P0, L0, C0 = Rectangle(0,0, 10, 5, lc=0.5)      # vonkajsi priestor

lc = 0.01
P1, L1, C1 = Rectangle(0,0, 3, 1, lc=0.1)       # vnutorny objekt

gmsh.model.geo.addPlaneSurface([C1])           # generovanie vnutorneho objektu
gmsh.model.geo.addPlaneSurface([C0, C1])       # generovanie zostavy

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)

gmsh.write("./data/demo_api_02.vtk")
gmsh.write("./data/demo_api_02.msh") 
gmsh.write("./data/demo_api_02.geo_unrolled")   # zapis geo - suboru

#import os
#_ = os.system('gmsh ./data/demo_api_02.msh &')

Vygenerovana mriezka modelu. 
Celkovú zostavu treba generovať ako poslednú, inak sa pre vnútorný objekt vygeneruje sanostatná mriežka a tieto sa prekryjú.

<img src="./img/obr_gmsh_02.png" width="600">

### <font color='purple'> <b>  Príklad 3. Definovanie susedných oblastí  </b></font>

Dve oblasti musia mať spoločnú hranicu, nemôžeme vtvoriť dve hranice, ktoré sa prekrývajú.Pre definíciu môžeme použiť už vytvorené línie a body zo susedných oblastí.

In [3]:
%reset -sf 
from utils.utils import *
gmsh.option.setNumber("General.Terminal",0)

gmsh.model.add("Hranice")            
P0, L0, C0 = Rectangle( 0, 0, 10, 5)      # vonkajsi priestor
P1, L1, C1 = Rectangle(-2, 0,  2, 1) 
P2, L2, C2 = Rectangle( 2, 0,  2, 1)

# vytvorenie prostrednej oblasti
p1 = Point(0, 2, lc=0.05)
p2 = Point(0,-2, lc=0.05)

C4 = gmsh.model.geo.addCurveLoop([   # orientovana krivka !!!
    -L1[1],
    Line(P1[1], p1),
    Line(p1, P2[0]),
    -L2[3],
    Line(P2[3],p2),
    Line(p2, P1[2])
    ])


q = gmsh.model.geo.addPlaneSurface([C4])
w = gmsh.model.geo.addPlaneSurface([C0, C1, C2, C4])

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)
gmsh.write("./data/demo_api_03.vtk")
gmsh.write("./data/demo_api_03.msh") 
gmsh.write("./data/demo_api_03.geo_unrolled")   # zapis geo - suboru

#import os
3_ = os.system('gmsh ./data/demo_api_03.msh &')

<img src="./img/obr_gmsh_03.png" width="600">

### <font color='purple'> <b>  Príklad 4. Definovanie fyzických entít </b></font>

Pre simulačný model je potrebné definovať fyzické časti, ktoré reprezentujú elektródy, materiál a pod. Pre definíciu oblastí v 2D sa používa krivka, ktorou je ohraničená oblasť.


In [29]:
%reset -sf 
from utils.utils import *
gmsh.option.setNumber("General.Terminal",0)

gmsh.model.add("Elektroda")            
P0, L0, C0 = Rectangle(0, 0, 10, 5, lc=0.5)       # vonkajsi priestor
P1, L1, C1 = Rectangle(0, 0,  5, 1, lc=0.1)       # vnutorny objekt

w = gmsh.model.geo.addPlaneSurface([C0, C1])

dim=2
space = gmsh.model.geo.addPhysicalGroup(dim, [w])      # simulacny priestor
gmsh.model.setPhysicalName(dim, space, "Space")

dim=1
b = gmsh.model.geo.addPhysicalGroup(dim, L0)           # okraj priestoru
gmsh.model.setPhysicalName(dim, b, "Boundary")

e = gmsh.model.geo.addPhysicalGroup(dim, L1)            # elektroda
gmsh.model.setPhysicalName(dim, e, "Elektroda")

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)
gmsh.write("./data/demo_api_04.vtk")
gmsh.write("./data/demo_api_04.msh") 
gmsh.write("./data/demo_api_04.geo_unrolled")   # zapis geo - suboru

#import os
#_ = os.system('gmsh ./data/demo_api_04.msh &')

<img src="./img/obr_gmsh_04.png" width="600">

------------